# Function

In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import urllib

def rotoguru_baseball_pts(mth, day, yr):
    url = "http://rotoguru1.com/cgi-bin/byday.pl?game=dk&month={}&day={}&year={}".format(mth, day, yr)
    
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page,'html.parser')
    
    header = soup.find_all('u')
    head = []
    for var in header:
        cols = var.find_all('b')
        cols = [ele.text.strip() for ele in cols]
        head.append(cols)
        
    head = head[:7]
    head[0] = ['Pos']
    head.insert(1, ['Player'])

    flat_head = [item for sublist in head for item in sublist]
    
    opp = soup.find_all('tr')
    oppo = []
    for ele in opp:
        op = ele.find_all('td', attrs={'nowrap':""})
        op = [ele.text for ele in op]
        oppo.append(op)

    opponent = oppo[9:-2]
    while ['Jump to: \xa0 \xa0 Pitchers \xa0 | \xa0 Hitters \xa0 | '] in opponent:
        opponent.remove(['Jump to: \xa0 \xa0 Pitchers \xa0 | \xa0 Hitters \xa0 | '])
    while ['Hitters '] in opponent:
        opponent.remove(['Hitters '])
    
    dat_df = pd.DataFrame(data=opponent, columns=flat_head)
    return(dat_df)

# Example

In [ ]:
import numpy as np

d = np.arange(1, 31, 1)

june_df = pd.DataFrame()

for i in d:
    temp_df = rotoguru_baseball_pts(mth=6, day=i, yr=2019)
    june_df = pd.concat([june_df, temp_df], ignore_index=True) 
    
june_df

june_df['Points'] = june_df['Points'].astype(float)

june_df.to_csv('C:/Users/finnxm/DK_June_pts.csv', index = None, header=True)